# Home Depot Product Search Relevance
The challenge is to predict a relevance score for the provided combinations of search terms and products. To create the ground truth labels, Home Depot has crowdsourced the search/product pairs to multiple human raters.

## LabGraph Create
This notebook uses the LabGraph create machine learning iPython module. You need a personal licence to run this code.

In [2]:
import graphlab as gl

### Load training, testing set and product descriptions

In [3]:
train = gl.SFrame.read_csv("../data/train.csv")

[INFO] This non-commercial license of GraphLab Create is assigned to thomasv1000@hotmail.fr and will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-15997 - Server binary: /Users/tjaskula/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454370987.log
[INFO] GraphLab Server Version: 1.8.1


PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.138959 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/train.csv
PROGRESS: Parsing completed. Parsed 74067 lines in 0.160986 secs.


In [4]:
train

id,product_uid,product_title,search_term,relevance
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0


In [5]:
test = gl.SFrame.read_csv("../data/test.csv")

PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/test.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.241514 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/test.csv
PROGRESS: Parsing completed. Parsed 166693 lines in 0.33245 secs.


In [6]:
test

id,product_uid,product_title,search_term
1,100001,Simpson Strong-Tie12-Gauge Angle ...,90 degree bracket
4,100001,Simpson Strong-Tie12-Gauge Angle ...,metal l brackets
5,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson sku able
6,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson strong ties
7,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson strong tie hcc668
8,100001,Simpson Strong-Tie12-Gauge Angle ...,wood connectors
10,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,bath and shower kit
11,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,bath drain kit
12,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,one piece tub shower
13,100004,Grape Solar 265-WattPolycrystalline Solar ...,solar panel


In [7]:
desc = gl.SFrame.read_csv("../data/product_descriptions.csv")

PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/product_descriptions.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.53035 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 61134 lines. Lines per second: 57649.8
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/product_descriptions.csv
PROGRESS: Parsing completed. Parsed 124428 lines in 1.64014 secs.


In [8]:
desc

product_uid,product_description
100001,"Not only do angles makejoints stronger, they ..."
100002,BEHR Premium TexturedDECKOVER is an innova ...
100003,Classic architecturemeets contemporary de ...
100004,The Grape Solar 265-WattPolycrystalline PV Solar ...
100005,Update your bathroom withthe Delta Vero Single- ...
100006,Achieving deliciousresults is almost ...
100007,The Quantum Adjustable2-Light LED Black ...
100008,The Teks #10 x 1-1/2 in.Zinc-Plated Steel Was ...
100009,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...
100010,Valley View IndustriesMetal Stakes (4-Pack) ...


In [9]:
train['relevance'].sketch_summary()


+--------------------+----------------+----------+
|        item        |     value      | is exact |
+--------------------+----------------+----------+
|       Length       |     74067      |   Yes    |
|        Min         |      1.0       |   Yes    |
|        Max         |      3.0       |   Yes    |
|        Mean        | 2.38163379103  |   Yes    |
|        Sum         |   176400.47    |   Yes    |
|      Variance      | 0.285135007425 |   Yes    |
| Standard Deviation | 0.533980343669 |   Yes    |
|  # Missing Values  |       0        |   Yes    |
|  # unique values   |       13       |    No    |
+--------------------+----------------+----------+

Most frequent items:
+-------+-------+-------+-------+-------+------+------+------+-----+------+------+
| value |  3.0  |  2.33 |  2.67 |  2.0  | 1.67 | 1.33 | 1.0  | 2.5 | 2.75 | 2.25 |
+-------+-------+-------+-------+-------+------+------+------+-----+------+------+
| count | 19125 | 16060 | 15202 | 11730 | 6780 | 3006 | 2105 |  1

In [11]:
gl.canvas.set_target('ipynb')

In [13]:
train.show()

In [14]:
train = train.join(desc, on = 'product_uid', how = 'left')

In [15]:
train

id,product_uid,product_title,search_term,relevance,product_description
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0,"Not only do angles makejoints stronger, they ..."
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5,"Not only do angles makejoints stronger, they ..."
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium TexturedDECKOVER is an innova ...
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33,Update your bathroom withthe Delta Vero Single- ...
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67,Update your bathroom withthe Delta Vero Single- ...
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0,Achieving deliciousresults is almost ...
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67,Achieving deliciousresults is almost ...
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0,Achieving deliciousresults is almost ...
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67,The Quantum Adjustable2-Light LED Black ...
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...
